In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
import sys
import matplotlib.pyplot as plt
import matplotlib
import h5py
import pandas as pd
from PIL import Image
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../resnet/")
from resnet import ResNet
import resnet

In [30]:
model = ResNet(3)

../../resnet/resnet.py:138: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight)


In [31]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
pytorch_total_params

270410

In [5]:
f = h5py.File("../../smoothed_geometry/example/weights/resnet20_ssr1_3.h5", "r")


In [6]:
i = 0
for layer, group in f.items():
    print(layer)
    for p_name in group.keys():
        param = group[p_name]
        for k_name in param.keys():
            print(k_name)
            print(param[k_name].shape)
            i+=1
print(i)

activation_1
activation_10
activation_11
activation_12
activation_13
activation_14
activation_15
activation_16
activation_17
activation_18
activation_19
activation_2
activation_20
activation_3
activation_4
activation_5
activation_6
activation_7
activation_8
activation_9
add_1
add_2
add_3
add_4
add_5
add_6
add_7
add_8
add_9
average_pooling2d_1
batch_normalization_1
beta:0
(16,)
gamma:0
(16,)
moving_mean:0
(16,)
moving_variance:0
(16,)
batch_normalization_10
beta:0
(32,)
gamma:0
(32,)
moving_mean:0
(32,)
moving_variance:0
(32,)
batch_normalization_11
beta:0
(32,)
gamma:0
(32,)
moving_mean:0
(32,)
moving_variance:0
(32,)
batch_normalization_12
beta:0
(32,)
gamma:0
(32,)
moving_mean:0
(32,)
moving_variance:0
(32,)
batch_normalization_13
beta:0
(32,)
gamma:0
(32,)
moving_mean:0
(32,)
moving_variance:0
(32,)
batch_normalization_14
beta:0
(64,)
gamma:0
(64,)
moving_mean:0
(64,)
moving_variance:0
(64,)
batch_normalization_15
beta:0
(64,)
gamma:0
(64,)
moving_mean:0
(64,)
moving_variance:0
(64,

In [43]:
f["conv2d_10"]["conv2d_10"]["kernel:0"][:].T.shape

(32, 16, 1, 1)

In [8]:
f["batch_normalization_1"]["batch_normalization_1"]["beta:0"][:]

array([0.12704657, 0.88938755, 0.1391233 , 0.16750452, 0.79546124,
       1.0130746 , 0.40594456, 1.0238892 , 1.0054772 , 1.5747428 ,
       0.34093332, 1.3006516 , 1.0709275 , 0.2351042 , 0.8880941 ,
       0.5203876 ], dtype=float32)

In [9]:
model

ResNet(
  (convIn): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bnIn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (stack1): ModuleList(
    (0): block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (downsample): AvgPool2d(kernel_size=1, stride=2, padding=0)
    )
    (1): block(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2)

In [43]:
model.stack3b[1].conv2.weight.size()

torch.Size([64, 64, 3, 3])

In [35]:
## load the weights
model.convIn.weight.data = torch.tensor(f["conv2d_1"]["conv2d_1"]["kernel:0"][:].T)
model.convIn.bias.data = torch.tensor(f["conv2d_1"]["conv2d_1"]["bias:0"][:])
#
model.bnIn.bias.data = torch.tensor(f["batch_normalization_1"]["batch_normalization_1"]["beta:0"][:])
model.bnIn.weight.data = torch.tensor(f["batch_normalization_1"]["batch_normalization_1"]["gamma:0"][:])
model.bnIn.running_mean.data = torch.tensor(f["batch_normalization_1"]["batch_normalization_1"]["moving_mean:0"][:])
model.bnIn.running_var.data = torch.tensor(f["batch_normalization_1"]["batch_normalization_1"]["moving_variance:0"][:])
# stack1
s1 = model.stack1
s1[0].conv1.weight.data = torch.tensor(f["conv2d_2"]["conv2d_2"]["kernel:0"][:].T)
s1[0].conv1.bias.data = torch.tensor(f["conv2d_2"]["conv2d_2"]["bias:0"][:])
#
s1[0].bn1.bias.data = torch.tensor(f["batch_normalization_2"]["batch_normalization_2"]["beta:0"][:])
s1[0].bn1.weight.data = torch.tensor(f["batch_normalization_2"]["batch_normalization_2"]["gamma:0"][:])
s1[0].bn1.running_mean.data = torch.tensor(f["batch_normalization_2"]["batch_normalization_2"]["moving_mean:0"][:])
s1[0].bn1.running_var.data = torch.tensor(f["batch_normalization_2"]["batch_normalization_2"]["moving_variance:0"][:])
#
s1[0].conv2.weight.data = torch.tensor(f["conv2d_3"]["conv2d_3"]["kernel:0"][:].T)
s1[0].conv2.bias.data = torch.tensor(f["conv2d_3"]["conv2d_3"]["bias:0"][:])
#
s1[0].bn2.bias.data = torch.tensor(f["batch_normalization_3"]["batch_normalization_3"]["beta:0"][:])
s1[0].bn2.weight.data = torch.tensor(f["batch_normalization_3"]["batch_normalization_3"]["gamma:0"][:])
s1[0].bn2.running_mean.data = torch.tensor(f["batch_normalization_3"]["batch_normalization_3"]["moving_mean:0"][:])
s1[0].bn2.running_var.data = torch.tensor(f["batch_normalization_3"]["batch_normalization_3"]["moving_variance:0"][:])
##########
s1[1].conv1.weight.data = torch.tensor(f["conv2d_4"]["conv2d_4"]["kernel:0"][:].T)
s1[1].conv1.bias.data = torch.tensor(f["conv2d_4"]["conv2d_4"]["bias:0"][:])
#
s1[1].bn1.bias.data = torch.tensor(f["batch_normalization_4"]["batch_normalization_4"]["beta:0"][:])
s1[1].bn1.weight.data = torch.tensor(f["batch_normalization_4"]["batch_normalization_4"]["gamma:0"][:])
s1[1].bn1.running_mean.data = torch.tensor(f["batch_normalization_4"]["batch_normalization_4"]["moving_mean:0"][:])
s1[1].bn1.running_var.data = torch.tensor(f["batch_normalization_4"]["batch_normalization_4"]["moving_variance:0"][:])
#
s1[1].conv2.weight.data = torch.tensor(f["conv2d_5"]["conv2d_5"]["kernel:0"][:].T)
s1[1].conv2.bias.data = torch.tensor(f["conv2d_5"]["conv2d_5"]["bias:0"][:])
#
s1[1].bn2.bias.data = torch.tensor(f["batch_normalization_5"]["batch_normalization_5"]["beta:0"][:])
s1[1].bn2.weight.data = torch.tensor(f["batch_normalization_5"]["batch_normalization_5"]["gamma:0"][:])
s1[1].bn2.running_mean.data = torch.tensor(f["batch_normalization_5"]["batch_normalization_5"]["moving_mean:0"][:])
s1[1].bn2.running_var.data = torch.tensor(f["batch_normalization_5"]["batch_normalization_5"]["moving_variance:0"][:])
###########
s1[2].conv1.weight.data = torch.tensor(f["conv2d_6"]["conv2d_6"]["kernel:0"][:].T)
s1[2].conv1.bias.data = torch.tensor(f["conv2d_6"]["conv2d_6"]["bias:0"][:])
#
s1[2].bn1.bias.data = torch.tensor(f["batch_normalization_6"]["batch_normalization_6"]["beta:0"][:])
s1[2].bn1.weight.data = torch.tensor(f["batch_normalization_6"]["batch_normalization_6"]["gamma:0"][:])
s1[2].bn1.running_mean.data = torch.tensor(f["batch_normalization_6"]["batch_normalization_6"]["moving_mean:0"][:])
s1[2].bn1.running_var.data = torch.tensor(f["batch_normalization_6"]["batch_normalization_6"]["moving_variance:0"][:])
#
s1[2].conv2.weight.data = torch.tensor(f["conv2d_7"]["conv2d_7"]["kernel:0"][:].T)
s1[2].conv2.bias.data = torch.tensor(f["conv2d_7"]["conv2d_7"]["bias:0"][:])
#
s1[2].bn2.bias.data = torch.tensor(f["batch_normalization_7"]["batch_normalization_7"]["beta:0"][:])
s1[2].bn2.weight.data = torch.tensor(f["batch_normalization_7"]["batch_normalization_7"]["gamma:0"][:])
s1[2].bn2.running_mean.data = torch.tensor(f["batch_normalization_7"]["batch_normalization_7"]["moving_mean:0"][:])
s1[2].bn2.running_var.data = torch.tensor(f["batch_normalization_7"]["batch_normalization_7"]["moving_variance:0"][:])
# stack 2a
s2a = model.stack2a
s2a.conv1.weight.data = torch.tensor(f["conv2d_8"]["conv2d_8"]["kernel:0"][:].T)
s2a.conv1.bias.data = torch.tensor(f["conv2d_8"]["conv2d_8"]["bias:0"][:])
#
s2a.bn1.bias.data = torch.tensor(f["batch_normalization_8"]["batch_normalization_8"]["beta:0"][:])
s2a.bn1.weight.data = torch.tensor(f["batch_normalization_8"]["batch_normalization_8"]["gamma:0"][:])
s2a.bn1.running_mean.data = torch.tensor(f["batch_normalization_8"]["batch_normalization_8"]["moving_mean:0"][:])
s2a.bn1.running_var.data = torch.tensor(f["batch_normalization_8"]["batch_normalization_8"]["moving_variance:0"][:])
#
s2a.conv2.weight.data = torch.tensor(f["conv2d_9"]["conv2d_9"]["kernel:0"][:].T)
s2a.conv2.bias.data = torch.tensor(f["conv2d_9"]["conv2d_9"]["bias:0"][:])
#
s2a.bn2.bias.data = torch.tensor(f["batch_normalization_9"]["batch_normalization_9"]["beta:0"][:])
s2a.bn2.weight.data = torch.tensor(f["batch_normalization_9"]["batch_normalization_9"]["gamma:0"][:])
s2a.bn2.running_mean.data = torch.tensor(f["batch_normalization_9"]["batch_normalization_9"]["moving_mean:0"][:])
s2a.bn2.running_var.data = torch.tensor(f["batch_normalization_9"]["batch_normalization_9"]["moving_variance:0"][:])
# stack 2b
s2b = model.stack2b
s2b[0].conv1.weight.data = torch.tensor(f["conv2d_10"]["conv2d_10"]["kernel:0"][:].T)
s2b[0].conv1.bias.data = torch.tensor(f["conv2d_10"]["conv2d_10"]["bias:0"][:])
#
s2b[0].bn1.bias.data = torch.tensor(f["batch_normalization_10"]["batch_normalization_10"]["beta:0"][:])
s2b[0].bn1.weight.data = torch.tensor(f["batch_normalization_10"]["batch_normalization_10"]["gamma:0"][:])
s2b[0].bn1.running_mean.data = torch.tensor(f["batch_normalization_10"]["batch_normalization_10"]["moving_mean:0"][:])
s2b[0].bn1.running_var.data = torch.tensor(f["batch_normalization_10"]["batch_normalization_10"]["moving_variance:0"][:])
#
s2b[0].conv2.weight.data = torch.tensor(f["conv2d_11"]["conv2d_11"]["kernel:0"][:].T)
s2b[0].conv2.bias.data = torch.tensor(f["conv2d_11"]["conv2d_11"]["bias:0"][:])
#
s2b[0].bn2.bias.data = torch.tensor(f["batch_normalization_11"]["batch_normalization_11"]["beta:0"][:])
s2b[0].bn2.weight.data = torch.tensor(f["batch_normalization_11"]["batch_normalization_11"]["gamma:0"][:])
s2b[0].bn2.running_mean.data = torch.tensor(f["batch_normalization_11"]["batch_normalization_11"]["moving_mean:0"][:])
s2b[0].bn2.running_var.data = torch.tensor(f["batch_normalization_11"]["batch_normalization_11"]["moving_variance:0"][:])
##########
s2b[1].conv1.weight.data = torch.tensor(f["conv2d_12"]["conv2d_12"]["kernel:0"][:].T)
s2b[1].conv1.bias.data = torch.tensor(f["conv2d_12"]["conv2d_12"]["bias:0"][:])
#
s2b[1].bn1.bias.data = torch.tensor(f["batch_normalization_12"]["batch_normalization_12"]["beta:0"][:])
s2b[1].bn1.weight.data = torch.tensor(f["batch_normalization_12"]["batch_normalization_12"]["gamma:0"][:])
s2b[1].bn1.running_mean.data = torch.tensor(f["batch_normalization_12"]["batch_normalization_12"]["moving_mean:0"][:])
s2b[1].bn1.running_var.data = torch.tensor(f["batch_normalization_12"]["batch_normalization_12"]["moving_variance:0"][:])
#
s2b[1].conv2.weight.data = torch.tensor(f["conv2d_13"]["conv2d_13"]["kernel:0"][:].T)
s2b[1].conv2.bias.data = torch.tensor(f["conv2d_13"]["conv2d_13"]["bias:0"][:])
#
s2b[1].bn2.bias.data = torch.tensor(f["batch_normalization_13"]["batch_normalization_13"]["beta:0"][:])
s2b[1].bn2.weight.data = torch.tensor(f["batch_normalization_13"]["batch_normalization_13"]["gamma:0"][:])
s2b[1].bn2.running_mean.data = torch.tensor(f["batch_normalization_13"]["batch_normalization_13"]["moving_mean:0"][:])
s2b[1].bn2.running_var.data = torch.tensor(f["batch_normalization_13"]["batch_normalization_13"]["moving_variance:0"][:])
# stack 3a
s3a = model.stack3a
s3a.conv1.weight.data = torch.tensor(f["conv2d_15"]["conv2d_15"]["kernel:0"][:].T)
s3a.conv1.bias.data = torch.tensor(f["conv2d_15"]["conv2d_15"]["bias:0"][:])
#
s3a.bn1.bias.data = torch.tensor(f["batch_normalization_14"]["batch_normalization_14"]["beta:0"][:])
s3a.bn1.weight.data = torch.tensor(f["batch_normalization_14"]["batch_normalization_14"]["gamma:0"][:])
s3a.bn1.running_mean.data = torch.tensor(f["batch_normalization_14"]["batch_normalization_14"]["moving_mean:0"][:])
s3a.bn1.running_var.data = torch.tensor(f["batch_normalization_14"]["batch_normalization_14"]["moving_variance:0"][:])
#
s3a.conv2.weight.data = torch.tensor(f["conv2d_16"]["conv2d_16"]["kernel:0"][:].T)
s3a.conv2.bias.data = torch.tensor(f["conv2d_16"]["conv2d_16"]["bias:0"][:])
#
s3a.bn2.bias.data = torch.tensor(f["batch_normalization_15"]["batch_normalization_15"]["beta:0"][:])
s3a.bn2.weight.data = torch.tensor(f["batch_normalization_15"]["batch_normalization_15"]["gamma:0"][:])
s3a.bn2.running_mean.data = torch.tensor(f["batch_normalization_15"]["batch_normalization_15"]["moving_mean:0"][:])
s3a.bn2.running_var.data = torch.tensor(f["batch_normalization_15"]["batch_normalization_15"]["moving_variance:0"][:])
# stack 3b
s3b = model.stack3b
s3b[0].conv1.weight.data = torch.tensor(f["conv2d_18"]["conv2d_18"]["kernel:0"][:].T)
s3b[0].conv1.bias.data = torch.tensor(f["conv2d_18"]["conv2d_18"]["bias:0"][:])
#
s3b[0].bn1.bias.data = torch.tensor(f["batch_normalization_16"]["batch_normalization_16"]["beta:0"][:])
s3b[0].bn1.weight.data = torch.tensor(f["batch_normalization_16"]["batch_normalization_16"]["gamma:0"][:])
s3b[0].bn1.running_mean.data = torch.tensor(f["batch_normalization_16"]["batch_normalization_16"]["moving_mean:0"][:])
s3b[0].bn1.running_var.data = torch.tensor(f["batch_normalization_16"]["batch_normalization_16"]["moving_variance:0"][:])
#
s3b[0].conv2.weight.data = torch.tensor(f["conv2d_19"]["conv2d_19"]["kernel:0"][:].T)
s3b[0].conv2.bias.data = torch.tensor(f["conv2d_19"]["conv2d_19"]["bias:0"][:])
#
s3b[0].bn2.bias.data = torch.tensor(f["batch_normalization_17"]["batch_normalization_17"]["beta:0"][:])
s3b[0].bn2.weight.data = torch.tensor(f["batch_normalization_17"]["batch_normalization_17"]["gamma:0"][:])
s3b[0].bn2.running_mean.data = torch.tensor(f["batch_normalization_17"]["batch_normalization_17"]["moving_mean:0"][:])
s3b[0].bn2.running_var.data = torch.tensor(f["batch_normalization_17"]["batch_normalization_17"]["moving_variance:0"][:])
##########
s3b[1].conv1.weight.data = torch.tensor(f["conv2d_20"]["conv2d_20"]["kernel:0"][:].T)
s3b[1].conv1.bias.data = torch.tensor(f["conv2d_20"]["conv2d_20"]["bias:0"][:])
#
s3b[1].bn1.bias.data = torch.tensor(f["batch_normalization_18"]["batch_normalization_18"]["beta:0"][:])
s3b[1].bn1.weight.data = torch.tensor(f["batch_normalization_18"]["batch_normalization_18"]["gamma:0"][:])
s3b[1].bn1.running_mean.data = torch.tensor(f["batch_normalization_18"]["batch_normalization_18"]["moving_mean:0"][:])
s3b[1].bn1.running_var.data = torch.tensor(f["batch_normalization_18"]["batch_normalization_18"]["moving_variance:0"][:])
#
s3b[1].conv2.weight.data = torch.tensor(f["conv2d_21"]["conv2d_21"]["kernel:0"][:].T)
s3b[1].conv2.bias.data = torch.tensor(f["conv2d_21"]["conv2d_21"]["bias:0"][:])
#
s3b[1].bn2.bias.data = torch.tensor(f["batch_normalization_19"]["batch_normalization_19"]["beta:0"][:])
s3b[1].bn2.weight.data = torch.tensor(f["batch_normalization_19"]["batch_normalization_19"]["gamma:0"][:])
s3b[1].bn2.running_mean.data = torch.tensor(f["batch_normalization_19"]["batch_normalization_19"]["moving_mean:0"][:])
s3b[1].bn2.running_var.data = torch.tensor(f["batch_normalization_19"]["batch_normalization_19"]["moving_variance:0"][:])

In [36]:
model = model.eval()

In [100]:
# Normalisation parameters fo CIFAR10
means = [0.4918687901200927, 0.49185976472299225, 0.4918583862227116]
stds  = [0.24697121702736, 0.24696766978537033, 0.2469719877121087]

normalize = transforms.Normalize(
    mean=means,
    std=stds,
)

train_transform = transforms.Compose([
    # 4 pixels are padded on each side, 
    transforms.Pad(4),
    # a 32×32 crop is randomly sampled from the 
    # padded image or its horizontal flip.
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    normalize
])

test_transform = transforms.Compose([
    # For testing, we only evaluate the single 
    # view of the original 32×32 image.
    transforms.ToTensor(),
    normalize
])

In [37]:
cifar_test = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, 
                                          transform=transforms.ToTensor())

Files already downloaded and verified


In [38]:
test_loader = torch.utils.data.DataLoader(
        cifar_test,
        batch_size=32,
        shuffle=False
    )

In [39]:
def accuracy(predicted_logits, reference):
    """Compute the ratio of correctly predicted labels"""
    labels = torch.argmax(predicted_logits, 1)
    correct_predictions = labels.eq(reference)
    return correct_predictions.sum().float() / correct_predictions.nelement()

criterion = torch.nn.CrossEntropyLoss()

test_losses = []
test_accs = []
weights_for_avg = []
for batch_x, batch_y in test_loader:
#     batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    prediction = model(batch_x)
    loss = criterion(prediction, batch_y)
    acc = accuracy(prediction, batch_y)
    test_accs.append(acc)
    test_losses.append(loss)
    weights_for_avg.append(len(batch_x))

RuntimeError: Given groups=1, weight of size [32, 16, 1, 1], expected input[32, 32, 16, 16] to have 16 channels, but got 32 channels instead

In [33]:
np.average(test_accs, weights=weights_for_avg)

0.0955

In [22]:
model.stack3a.conv2

Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [8]:
i = 0
for child_name, child in model.named_children():
    if isinstance(child, nn.BatchNorm2d):
        i = i +1
        print(child)
    else:
        if isinstance(child, nn.ModuleList):
            for _, child1 in child.named_children():
                if isinstance(child1, resnet.block):
                    for _, child2 in child1.named_children():
                        if isinstance(child2, nn.BatchNorm2d):
                            print(child2)
                            i = i+1
        elif isinstance(child, resnet.block):
            for _, child3 in child.named_children():
                if isinstance(child3, nn.BatchNorm2d):
                    print(child3)
                    i = i+1
        else:
            pass
                
    

BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
BatchNorm2d(32, eps=1e-05, momentum=0.1,